In [16]:
import os
import boto3
import pandas as pd
import numpy as np
import sagemaker as sm
from io import StringIO

bucket = '1s-gary'
data_key = 'aiml-blackbelt-2021/rawdata/PS_20174392719_1491204439457_log.csv'
data_model_drift_key = 'aiml-blackbelt-2021/rawdata/PS_20174392719_1491204439457_log_model_drift.csv'
whats_in_here = 'aiml-blackbelt-2021/MLOpsData/PS_20174392719_1491204439457_log_model_drift_fe_0.csv'

s3_client = boto3.client('s3')
csv_obj = s3_client.get_object(Bucket=bucket, Key=whats_in_here)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))
model_data = pd.concat(
    [
        df["isFraud"],
        df.drop(["isFraud"], axis=1),
    ],
    axis=1,
)


In [17]:
model_data

,isFraud,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,deltaOrig,deltaDest,amount_previous,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,type_previous_CASH_OUT,type_previous_DEBIT,type_previous_PAYMENT,type_previous_TRANSFER
0,0,1864.28,21249.00,19384.72,0.00,0.00,-1864.28,0.00,9839.64,0,0,1,0,0,0,1,0
1,1,181.00,181.00,0.00,0.00,0.00,-181.00,0.00,1864.28,0,0,0,1,0,0,1,0
2,1,181.00,181.00,0.00,21182.00,0.00,-181.00,-21182.00,181.00,1,0,0,0,0,0,0,1
3,0,11668.14,41554.00,29885.86,0.00,0.00,-11668.14,0.00,181.00,0,0,1,0,1,0,0,0
4,0,7817.71,53860.00,46042.29,0.00,0.00,-7817.71,0.00,11668.14,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617632,0,15219.96,17175.00,1955.04,0.00,0.00,-15219.96,0.00,235623.40,0,0,1,0,1,0,0,0
1617633,0,278384.75,0.00,0.00,0.00,278384.75,0.00,278384.75,15219.96,1,0,0,0,0,0,1,0
1617634,0,13597.57,0.00,0.00,0.00,0.00,0.00,0.00,278384.75,0,0,1,0,1,0,0,0
1617635,0,342970.21,78745.46,0.00,12320145.63,12663115.84,-78745.46,342970.21,13597.57,1,0,0,0,0,0,1,0


In [14]:
churn_data_key = 'sagemaker/DEMO-xgboost-churn/data/RawData.csv'
bucket = 'sagemaker-us-west-2-842337631775'
s3_client = boto3.client('s3')
csv_obj = s3_client.get_object(Bucket=bucket, Key=churn_data_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))
# drop the "Phone" feature column
df = df.drop(["Phone"], axis=1)

# Change the data type of "Area Code"
df["Area Code"] = df["Area Code"].astype(object)

# Drop several other columns
df = df.drop(["Day Charge", "Eve Charge", "Night Charge", "Intl Charge"], axis=1)

# Convert categorical variables into dummy/indicator variables.
model_data = pd.get_dummies(df)

# Create one binary classification target column
model_data = pd.concat(
    [
        model_data["Churn?_True."],
        model_data.drop(["Churn?_False.", "Churn?_True."], axis=1),
    ],
    axis=1,
)
df

,State,Account Length,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,Intl Calls,CustServ Calls,Churn?
0,PA,163,806,no,yes,300,8.162204,3,3.933035,4,4.065759,100,4.928160,6,3,True.
1,SC,15,836,yes,no,0,10.018993,4,2.325005,0,7.141040,200,3.221748,6,8,False.
2,MO,131,777,no,yes,300,4.708490,3,4.537466,3,5.363235,100,7.139023,2,4,False.
3,WY,75,878,yes,yes,700,1.268734,3,2.528748,5,3.773586,450,2.245779,6,6,False.
4,WY,146,878,yes,no,0,2.696177,3,6.015337,3,3.751673,250,6.905545,4,6,True.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,NH,4,787,yes,yes,800,10.862632,5,6.936164,1,4.921314,350,4.872570,8,9,False.
4996,SD,140,836,no,no,0,1.581127,8,7.377591,7,0.939932,300,6.938571,2,4,False.
4997,SC,32,836,no,yes,700,0.163836,5,5.841852,3,0.939469,450,4.388328,7,6,False.
4998,MA,142,776,yes,yes,600,2.034454,5,4.140554,3,6.076043,150,7.173376,3,7,True.


In [15]:
model_data

,Churn?_True.,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,...,Area Code_858,Area Code_866,Area Code_868,Area Code_876,Area Code_877,Area Code_878,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,1,163,300,8.162204,3,3.933035,4,4.065759,100,4.928160,...,0,0,0,0,0,0,1,0,0,1
1,0,15,0,10.018993,4,2.325005,0,7.141040,200,3.221748,...,0,0,0,0,0,0,0,1,1,0
2,0,131,300,4.708490,3,4.537466,3,5.363235,100,7.139023,...,0,0,0,0,0,0,1,0,0,1
3,0,75,700,1.268734,3,2.528748,5,3.773586,450,2.245779,...,0,0,0,0,0,1,0,1,0,1
4,1,146,0,2.696177,3,6.015337,3,3.751673,250,6.905545,...,0,0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,4,800,10.862632,5,6.936164,1,4.921314,350,4.872570,...,0,0,0,0,0,0,0,1,0,1
4996,0,140,0,1.581127,8,7.377591,7,0.939932,300,6.938571,...,0,0,0,0,0,0,1,0,1,0
4997,0,32,700,0.163836,5,5.841852,3,0.939469,450,4.388328,...,0,0,0,0,0,0,1,0,0,1
4998,1,142,600,2.034454,5,4.140554,3,6.076043,150,7.173376,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
def feature_engineer(df,drop_first=True):
    '''
    Take a raw dataframe and creates the features we want for machine learning
    '''
    # now do data engineering on df and then write out in steps.
    df['deltaOrig']=df['newbalanceOrig']-df['oldbalanceOrg']
    df['deltaDest']=df['newbalanceDest']-df['oldbalanceDest']
    # get lags; see analysis below
    df.loc[:,'amount_previous'] = df.amount.shift(1)
    df.loc[:,'type_previous'] = df.type.shift(1)
    df.drop(['nameOrig','nameDest','isFlaggedFraud','step'],axis=1,inplace=True)
    df1 = df.replace([np.inf, -np.inf], np.nan).dropna()
    df1 = pd.get_dummies(df1, drop_first=drop_first)
    return(df1)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics


dff = feature_engineer(df,drop_first=True)
print(dff.columns)

y = dff.pop('isFraud')
X = dff

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
def f1_score(contingency_matrix):
    if not contingency_matrix.shape==(2,2):
        print('F1 error')
    else:
        (TN,FP,FN,TP) = (contingency_matrix[0][0],contingency_matrix[0][1],contingency_matrix[1][0],contingency_matrix[1][1])
#        print(TN,FN,FP,TP)
        F1 = 2*TP / (2*TP + FP + FN)
# not quite sure what the uncertainty is but let's approximate it using the estimate here:
# https://stats.stackexchange.com/questions/363382/confidence-interval-of-precision-recall-and-f1-score
        sig_F1 = np.sqrt((F1)*(1-F1)/(4+TN+FP+FN+TP))
        return(F1,sig_F1)


In [ ]:
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

predictions = logisticRegr.predict(X_test)
cm = metrics.confusion_matrix(y_test, predictions)
print(f'\n logisticRegr')
print(cm,'\n F1 score:',f1_score(cm))

In [ ]:
def set_sample_weights(y_train):
    sample_weights = np.zeros(len(y_train))
    n_pos = sum(y_train)
    n_neg = len(y_train) - n_pos
    w_pos = (n_pos+n_neg)/(2*n_pos)
    w_neg = (n_pos+n_neg)/(2*n_neg)
    sample_weights[y_train == 0] = w_neg
    sample_weights[y_train == 1] = w_pos
    return(sample_weights)

In [ ]:
XGBClassifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)
sample_weights = set_sample_weights(y_train)
XGBClassifier.fit(X_train, y_train, sample_weight = sample_weights)


predictions = XGBClassifier.predict(X_test)
cm = metrics.confusion_matrix(y_test, predictions)
print(f'\n GradientBoostingClassifier')
print(cm,'\n F1 score:',f1_score(cm))